<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-to-Unix---awk-and-Makefiles" data-toc-modified-id="Introduction-to-Unix---awk-and-Makefiles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction to Unix - awk and Makefiles</a></span></li><li><span><a href="#Working-with-tabular-files:-Awk" data-toc-modified-id="Working-with-tabular-files:-Awk-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Working with tabular files: Awk</a></span><ul class="toc-item"><li><span><a href="#Example-of-tabular-file:-the-GFF3-format" data-toc-modified-id="Example-of-tabular-file:-the-GFF3-format-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example of tabular file: the GFF3 format</a></span></li><li><span><a href="#Basic-AWK-syntax:-filters" data-toc-modified-id="Basic-AWK-syntax:-filters-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Basic AWK syntax: filters</a></span><ul class="toc-item"><li><span><a href="#Exercise" data-toc-modified-id="Exercise-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Exercise</a></span></li></ul></li><li><span><a href="#Awk:-printing-columns-and-doing-operations" data-toc-modified-id="Awk:-printing-columns-and-doing-operations-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Awk: printing columns and doing operations</a></span><ul class="toc-item"><li><span><a href="#Exercise-(difficult)" data-toc-modified-id="Exercise-(difficult)-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Exercise (difficult)</a></span></li></ul></li><li><span><a href="#AWK:-searching-by-regular-expressions" data-toc-modified-id="AWK:-searching-by-regular-expressions-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>AWK: searching by regular expressions</a></span><ul class="toc-item"><li><span><a href="#Last-exercise!" data-toc-modified-id="Last-exercise!-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Last exercise!</a></span></li></ul></li></ul></li><li><span><a href="#Bonus:-Makefiles" data-toc-modified-id="Bonus:-Makefiles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bonus: Makefiles</a></span><ul class="toc-item"><li><span><a href="#Defining-pipelines-with-Makefiles" data-toc-modified-id="Defining-pipelines-with-Makefiles-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Defining pipelines with Makefiles</a></span></li><li><span><a href="#How-to-run-Makefile-rules" data-toc-modified-id="How-to-run-Makefile-rules-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>How to run Makefile rules</a></span></li><li><span><a href="#Dinner-time!!" data-toc-modified-id="Dinner-time!!-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Dinner time!!</a></span></li></ul></li></ul></div>

In [17]:
# Configuration - please ignore - this should not appear in the slideshow
alias grep='grep --color'
cd
cd workspace/Peb2019/exercises/

# Introduction to Unix - awk and Makefiles

**<font color='darkgreen'> Welcome to the Programming for Evolutionary Biology workshop!! </font>** 

Giovanni M. Dall'Olio and Alvaro Perdomo-Sabogal, 03/03/2019. All materials available here: https://github.com/dalloliogm/evop2019/archive/master.zip


In this fourth part we will use the **awk** command to excplore the contents of files, and some basic regular expressions.

Press space or down key to continue.

# Working with tabular files: Awk


The **awk** command allows to search and manipulate tabular files from the command line.

Imagine it as the equivalent of Excel/Calc for the command line. It allows to do search on specific columns of a file, to do numerical operations, or to change the order of the columns.

The advantage of a command-line tool over graphical software is that the memory footprint is much lower. So you can access and modify large files in a fraction of the time that it would take with Excel.

## Example of tabular file: the GFF3 format

The file genes/chr8.gff contains an example of file in the GFF3 format:

In [18]:
head genes/chr8.gff

##gff-version 3
##source-version rtracklayer 1.34.1
##date 2017-02-27
##genome-build .	hg19
chr8	rtracklayer	sequence_feature	18248755	18258723	.	+	.	gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	91970706	91997485	.	-	.	gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8	rtracklayer	sequence_feature	74332309	74353753	.	+	.	gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126


As you can see it is a tab-separated file, which we could easily read in Excel or Calc.

The **GFF** (General Feature Format) format specifications are defined [here](https://genome.ucsc.edu/FAQ/FAQformat.html#format3), but in short:

- the **col1**, **col4** and **col5** contain the chromosome name and genomic coordinates (start and end),
- the **col2** describes the tool or resource that generated the annotation,
- the **col3** describe the type of feature (e.g. gene, transcript, exon, TF binding site, Histone Acetylation mark, etc...
- the **col9** column contains several fields, separated by a semicolon


## Basic AWK syntax: filters

The basic AWK syntax is the following:


** awk 'filters {print statements}' filename**

Awk is quite smart at recognizing the field separator, and by default assumes they are separated by tabs.

Each column of the file can be referred to with the dollar sign followed by the number of column.

For example $2 refers to the second column, and so on.

The following code filters all the lines belonging to chromosome 8, between the coordinates 100000 and 200000:

In [20]:
# This prints all the lines on chromosome 8 and with chromosomal position between 10000 and 20000
# Don't be scared by this long statement, we will explain it step by step
# 
awk '$1=="chr8" && $4>100000 && $5<200000 ' genes/chr8.gff

chr8	rtracklayer	sequence_feature	182200	197339	.	+	.	gene_id=169270;symbol=ZNF596;exerc1=FALSE;ID=169270
chr8	rtracklayer	sequence_feature	116086	117024	.	-	.	gene_id=441308;symbol=OR4F21;exerc1=FALSE;ID=441308
chr8	rtracklayer	sequence_feature	158345	182318	.	-	.	gene_id=644128;symbol=RPL23AP53;exerc1=FALSE;ID=644128


### Exercise

Can you print all the lines between 5000000 and 10000000 ?

In [21]:
awk '$4 > 5000000 && $5 < 10000000 ' genes/chr8.gff


chr8	rtracklayer	sequence_feature	7143733	7212876	.	-	.	gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890
chr8	rtracklayer	sequence_feature	7215498	7220490	.	-	.	gene_id=100131980;symbol=ZNF705G;exerc1=TRUE;ID=100131980
chr8	rtracklayer	sequence_feature	7812535	7866277	.	+	.	gene_id=100132103;symbol=FAM66E;exerc1=TRUE;ID=100132103
chr8	rtracklayer	sequence_feature	7783859	7809935	.	+	.	gene_id=100132396;symbol=ZNF705B;exerc1=TRUE;ID=100132396
chr8	rtracklayer	sequence_feature	6261077	6264069	.	-	.	gene_id=100287015;symbol=LOC100287015;exerc1=TRUE;ID=100287015
chr8	rtracklayer	sequence_feature	7272385	7274354	.	-	.	gene_id=100289462;symbol=DEFB4B;exerc1=TRUE;ID=100289462
chr8	rtracklayer	sequence_feature	7946463	7946611	.	-	.	gene_id=100302186;symbol=MIR548I3;exerc1=TRUE;ID=100302186
chr8	rtracklayer	sequence_feature	6602685	6602765	.	+	.	gene_id=100616348;symbol=MIR4659A;exerc1=TRUE;ID=100616348
chr8	rtracklayer	sequence_feature	8905955	8906028	.	+	.	gene_id=100616350;symbol=MIR

## Awk: printing columns and doing operations

Awk also allows to print only specific columns, and do algebraic operations on them.

Remember that each column can be referred as \$1, \$2, \$3, etc...

For example the following code prints the first column, and the sum of the fourth and third. We can pipe the output to head or less, to make it easier to visualize:

In [8]:
awk '{print $1, $5-$4}' genes/chr8.gff | head


##gff-version 0
##source-version 0
##date 0
##genome-build 0
chr8 9968
chr8 75
chr8 85
chr8 81
chr8 26779
chr8 21444


Notice how this also prints the headers of the file. We can exclude these by adding a grep condition:

In [9]:
awk '{print $1, $5-$4, $9}' genes/chr8.gff | grep -v '^#' |  head

chr8 9968 gene_id=10;symbol=NAT2;exerc1=FALSE;ID=10
chr8 75 gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8 85 gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8 81 gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8 26779 gene_id=100127983;symbol=C8orf88;exerc1=FALSE;ID=100127983
chr8 21444 gene_id=100128126;symbol=STAU2-AS1;exerc1=FALSE;ID=100128126
chr8 12197 gene_id=100128338;symbol=FAM83H-AS1;exerc1=FALSE;ID=100128338
chr8 1835 gene_id=100128627;symbol=CDC42P3;exerc1=FALSE;ID=100128627
chr8 3282 gene_id=100128750;symbol=RBPMS-AS1;exerc1=FALSE;ID=100128750
chr8 69143 gene_id=100128890;symbol=FAM66B;exerc1=TRUE;ID=100128890


### Exercise (difficult)

Starting from the previous command, can you extract the gene symbol into a separate column?

Hints: pipe an additional awk statement after the first. Use the -F option to specify a different field separator.

In [10]:
awk '{print $1, $5-$4, $9}' genes/chr8.gff | grep -v '^#' | awk -F';' '{print $1, $2}' | head

chr8 9968 gene_id=10 symbol=NAT2
chr8 75 gene_id=100126309 symbol=MIR875
chr8 85 gene_id=100126338 symbol=MIR937
chr8 81 gene_id=100126351 symbol=MIR939
chr8 26779 gene_id=100127983 symbol=C8orf88
chr8 21444 gene_id=100128126 symbol=STAU2-AS1
chr8 12197 gene_id=100128338 symbol=FAM83H-AS1
chr8 1835 gene_id=100128627 symbol=CDC42P3
chr8 3282 gene_id=100128750 symbol=RBPMS-AS1
chr8 69143 gene_id=100128890 symbol=FAM66B


## AWK: searching by regular expressions

Awk can also be used to search by regular expression.

For example, the following code will print all the lines in which the symbol starts with "MIR":

In [11]:
awk '$9 ~ /symbol=MIR/ {print $0}' genes/chr8.gff 

chr8	rtracklayer	sequence_feature	100549014	100549089	.	-	.	gene_id=100126309;symbol=MIR875;exerc1=FALSE;ID=100126309
chr8	rtracklayer	sequence_feature	144895127	144895212	.	-	.	gene_id=100126338;symbol=MIR937;exerc1=FALSE;ID=100126338
chr8	rtracklayer	sequence_feature	145619364	145619445	.	-	.	gene_id=100126351;symbol=MIR939;exerc1=FALSE;ID=100126351
chr8	rtracklayer	sequence_feature	65285775	65295842	.	+	.	gene_id=100130155;symbol=MIR124-2HG;exerc1=FALSE;ID=100130155
chr8	rtracklayer	sequence_feature	128972879	128972941	.	+	.	gene_id=100302161;symbol=MIR1205;exerc1=FALSE;ID=100302161
chr8	rtracklayer	sequence_feature	10682883	10682953	.	-	.	gene_id=100302166;symbol=MIR1322;exerc1=FALSE;ID=100302166
chr8	rtracklayer	sequence_feature	129021144	129021202	.	+	.	gene_id=100302170;symbol=MIR1206;exerc1=FALSE;ID=100302170
chr8	rtracklayer	sequence_feature	129061398	129061484	.	+	.	gene_id=100302175;symbol=MIR1207;exerc1=FALSE;ID=100302175
chr8	rtracklayer	sequence_feature	128808208	12880827

### Last exercise!

Calculate the lenght of the gene POU5F1B.

Find the Gene whose gene_id is equal to that number.

In [12]:
awk '$9 ~ /POU5F1B/ {print $5-$4}' genes/chr8.gff 


1584


In [13]:
awk '$9 ~ /gene_id=1584/ {print $0}' genes/chr8.gff 

chr8	Well_done	Great_job	143953773	143961236	.	-	.	gene_id=1584;symbol=CYP11B1;exerc1=FALSE;ID=1584


# Bonus: Makefiles


You may have noticed that the unix_intro folder contains a file called Makefile. 
 
What is a Makefile? Let's have a look at its contents first: 

In [14]:
cd ..
head Makefile

# This is a Makefile, which will be explained later in the course.
# Please don't look at it yet :-)

publish: slides_bash commit
	echo "convert the slides to pdf, commit, and push to github"
	git push


test_exercises: start help ignorecase multiplefiles
generate_exercises: generate_grep generate_awk


Press space or the down key to continue

## Defining pipelines with Makefiles

Makefiles are a basic way to define pipelines of shell commands.

Nowadays there are more sophisticated tools available, but most of these are based on Makefiles.


A Makefile is a collection of "rules".

Each of these rules follows this basic syntax is:

```
target: prerequisites
    commands to execute
```

As you can see in the Makefile included, most of the rules allow to regenerate the exercise files, or to execute some commands without having to type them everytime.

For example, the rule "testrule" is associated to two echo commands.

## How to run Makefile rules

To execute a rule in the Makefile, simply type:

```
make [name of the rule]
```

For example:



In [25]:
make testrule

echo this is a Makefile rule
this is a Makefile rule
echo You can associate it to as many commands you want
You can associate it to as many commands you want


The program "make" will automatically detect any file named "Makefile" in the current directory, and execute any rule with the specific name.

Rules can also be nested together. For example the two rules "test_exercises" and "generate_exercises" at the beginning of the file are a way to call several other rules together.

## Dinner time!!
